In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [42]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
path='/content/drive/MyDrive/walmart Retail Data.xlsx'
data=pd.read_excel(path)
data.describe()

,Customer Age,Discount,Number of Records,Order Date,Order ID,Order Quantity,Product Base Margin,Profit,Row ID,Sales,Ship Date,Shipping Cost,Unit Price,Zip Code
count,7496.000000,8399.000000,8399.0,8399,8399.000000,8399.000000,8336.000000,8399.000000,8399.000000,8399.000000,8399,8399.000000,8399.000000,8399.000000
mean,54.542823,0.049671,1.0,2013-12-24 18:31:30.225026560,29965.179783,25.571735,0.512513,181.184422,4200.000000,1775.878179,2013-12-26 19:17:37.411596544,12.838557,89.346259,52839.139421
min,41.000000,0.000000,1.0,2012-01-01 00:00:00,3.000000,1.000000,0.350000,-14140.701600,1.000000,2.240000,2012-01-02 00:00:00,0.490000,0.990000,1001.000000
25%,47.000000,0.020000,1.0,2012-12-20 00:00:00,15011.500000,13.000000,0.380000,-83.315000,2100.500000,143.195000,2012-12-22 12:00:00,3.300000,6.480000,30337.000000
50%,53.000000,0.050000,1.0,2013-12-17 00:00:00,29857.000000,26.000000,0.520000,-1.500000,4200.000000,449.420000,2013-12-18 00:00:00,6.070000,20.990000,52732.000000
75%,61.000000,0.080000,1.0,2015-01-01 12:00:00,44596.000000,38.000000,0.590000,162.748000,6299.500000,1709.320000,2015-01-03 12:00:00,13.990000,85.990000,77577.000000
max,95.000000,0.250000,1.0,2015-12-30 00:00:00,59973.000000,50.000000,0.850000,27220.690000,8399.000000,89061.050000,2015-12-30 00:00:00,164.730000,6783.020000,99362.000000
std,9.519435,0.031823,0.0,NaN,17260.883447,14.481071,0.135589,1196.653326,2424.726789,3585.050525,NaN,17.264052,290.354383,28509.535635


simple imputer

In [43]:
data.isnull().sum()

,0
City,0
Customer Age,903
Customer Name,0
Customer Segment,0
Discount,0
Number of Records,0
Order Date,0
Order ID,0
Order Priority,0
Order Quantity,0


In [44]:
import pandas as pd
from sklearn.impute import SimpleImputer

In [45]:
imputer = SimpleImputer(strategy='mean')  # or 'median', 'most_frequent', 'constant'

In [46]:
# Assuming 'data' is your pandas DataFrame and 'Customer Age' is the column with missing values
imputer.fit(data[['Customer Age']])

SimpleImputer()

In [47]:
data['Customer Age'] = imputer.transform(data[['Customer Age']])

In [48]:
data.isnull().sum()


,0
City,0
Customer Age,0
Customer Name,0
Customer Segment,0
Discount,0
Number of Records,0
Order Date,0
Order ID,0
Order Priority,0
Order Quantity,0


In [49]:
print(data['Customer Segment'])

0       Small Business
1          Home Office
2            Corporate
3             Consumer
4            Corporate
             ...      
8394         Corporate
8395       Home Office
8396          Consumer
8397       Home Office
8398       Home Office
Name: Customer Segment, Length: 8399, dtype: object


nominal encoder

In [50]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

In [51]:
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore') # sparse=False for array output

In [52]:
# Assuming 'data' is your DataFrame and 'City' is the nominal feature
encoded_data = encoder.fit_transform(data[['Customer Segment']])

In [53]:
encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(['Customer Segment']))
data = pd.concat([data, encoded_df], axis=1)
data = data.drop('Customer Segment', axis=1) # Remove original column

In [54]:
print(encoded_df)

      Customer Segment_Consumer  Customer Segment_Corporate  \
0                           0.0                         0.0   
1                           0.0                         0.0   
2                           0.0                         1.0   
3                           1.0                         0.0   
4                           0.0                         1.0   
...                         ...                         ...   
8394                        0.0                         1.0   
8395                        0.0                         0.0   
8396                        1.0                         0.0   
8397                        0.0                         0.0   
8398                        0.0                         0.0   

      Customer Segment_Home Office  Customer Segment_Small Business  
0                              0.0                              1.0  
1                              1.0                              0.0  
2                              0.

Ordinal Encoder

In [55]:
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder

In [56]:
encoder = OrdinalEncoder(categories=[['Low', 'Medium', 'High']])  # Define the order of categories

In [58]:
# Assuming 'data' is your DataFrame and 'Order Priority' is the ordinal feature
# Check if 'Order Priority' exists in the DataFrame columns
if 'Order Priority' in data.columns:
    # Get unique values from 'Order Priority' column
    unique_values = data['Order Priority'].unique()

    # Update categories to include all unique values
    encoder = OrdinalEncoder(categories=[[val for val in unique_values]])

    data['Order Priority'] = encoder.fit_transform(data[['Order Priority']])
else:
    print("Column 'Order Priority' not found in DataFrame. Please check the column name.")

In [59]:
print(data['Order Priority'])

0       0.0
1       1.0
2       2.0
3       3.0
4       2.0
       ... 
8394    1.0
8395    2.0
8396    4.0
8397    2.0
8398    2.0
Name: Order Priority, Length: 8399, dtype: float64
